# 🤖 Modern NLP und Text Generation
### Woche 7, Notebook 2: Von Transformers bis Deployment

**Lernziele:**
- 🔤 **Modern NLP**: Transformer-basierte Modelle verstehen
- 🤗 **Hugging Face**: Pre-trained Models nutzen
- ✍️ **Text Generation**: GPT-Style Modelle anwenden
- 🌐 **NLP APIs**: Text-Services für Produktion
- 🚀 **Deployment**: NLP-Anwendungen bereitstellen

---

> 💡 **2025 Update**: Wir nutzen moderne Transformer-Modelle statt veralteter LSTM-Ansätze!

In [ ]:
# 📦 Modern NLP Setup
import warnings
warnings.filterwarnings('ignore')

# Basis Libraries
import pandas as pd
import numpy as np
from datetime import datetime
import logging

# Modern NLP
try:
    from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
    print("✅ Transformers verfügbar")
except ImportError:
    print("⚠️ Transformers nicht installiert")

# API Framework
try:
    from fastapi import FastAPI
    from pydantic import BaseModel
    print("✅ FastAPI verfügbar")
except ImportError:
    print("⚠️ FastAPI nicht installiert")

print(f"🚀 Modern NLP Setup abgeschlossen - {datetime.now().strftime('%H:%M:%S')}")

## 🔤 Modern NLP mit Transformers

### Warum Transformers statt LSTM?

| Aspekt | LSTM (Legacy) | Transformers (Modern) |
|--------|---------------|----------------------|
| **Performance** | Sequenziell, langsam | Parallel, schnell |
| **Kontext** | Begrenzt | Unbegrenzt |
| **Qualität** | Gut | Exzellent |
| **Pre-training** | Schwierig | Einfach |
| **Deployment** | Komplex | Streamlined |

In [ ]:
# 🤗 Hugging Face Text Generation Pipeline

class ModernTextGenerator:
    """
    Modern Text Generation mit Hugging Face Transformers
    """
    
    def __init__(self, model_name="distilgpt2"):
        self.model_name = model_name
        print(f"📥 Lade Modell: {model_name}...")
        
        # Text Generation Pipeline
        self.generator = pipeline(
            "text-generation",
            model=model_name,
            tokenizer=model_name,
            device=-1  # CPU
        )
        
        print("✅ Text Generator bereit!")
    
    def generate_text(self, prompt, max_length=100, num_return_sequences=1, temperature=0.7):
        """
        Text generieren basierend auf Prompt
        """
        try:
            results = self.generator(
                prompt,
                max_length=max_length,
                num_return_sequences=num_return_sequences,
                temperature=temperature,
                pad_token_id=self.generator.tokenizer.eos_token_id,
                do_sample=True
            )
            
            generated_texts = [result['generated_text'] for result in results]
            return generated_texts
            
        except Exception as e:
            print(f"❌ Generation Fehler: {str(e)}")
            return [f"Fehler bei der Generierung: {str(e)}"]

# Text Generator initialisieren
text_gen = ModernTextGenerator()

# Test Generation
test_prompt = "Once upon a time in a magical forest"
generated = text_gen.generate_text(test_prompt, max_length=80)

print("🧪 Test Generation:")
print(f"Prompt: {test_prompt}")
print(f"Generated: {generated[0]}")

### 🎯 Verschiedene NLP Tasks

In [ ]:
# 🔧 Multi-Task NLP Pipeline

class NLPTaskSuite:
    """
    Sammlung von NLP Tasks für praktische Anwendungen
    """
    
    def __init__(self):
        self.tasks = {}
        self._init_pipelines()
    
    def _init_pipelines(self):
        """Initialisiert verschiedene NLP Pipelines"""
        print("🔄 Initialisiere NLP Pipelines...")
        
        try:
            # Sentiment Analysis
            self.tasks['sentiment'] = pipeline("sentiment-analysis")
            print("✅ Sentiment Analysis geladen")
            
            # Text Summarization
            self.tasks['summarization'] = pipeline("summarization", model="facebook/bart-large-cnn")
            print("✅ Summarization geladen")
            
            # Question Answering
            self.tasks['qa'] = pipeline("question-answering")
            print("✅ Question Answering geladen")
            
        except Exception as e:
            print(f"⚠️ Einige Pipelines konnten nicht geladen werden: {str(e)}")
    
    def analyze_sentiment(self, text):
        """Sentiment Analysis"""
        if 'sentiment' not in self.tasks:
            return "Sentiment Pipeline nicht verfügbar"
        
        result = self.tasks['sentiment'](text)[0]
        return {
            'label': result['label'],
            'confidence': round(result['score'], 4)
        }
    
    def summarize_text(self, text, max_length=130):
        """Text Summarization"""
        if 'summarization' not in self.tasks:
            return "Summarization Pipeline nicht verfügbar"
        
        if len(text.split()) < 50:
            return "Text zu kurz für Zusammenfassung"
        
        result = self.tasks['summarization'](text, max_length=max_length, min_length=30)
        return result[0]['summary_text']
    
    def answer_question(self, context, question):
        """Question Answering"""
        if 'qa' not in self.tasks:
            return "QA Pipeline nicht verfügbar"
        
        result = self.tasks['qa'](question=question, context=context)
        return {
            'answer': result['answer'],
            'confidence': round(result['score'], 4)
        }

# NLP Suite initialisieren
nlp_suite = NLPTaskSuite()

# Tests
print("\n🧪 NLP Task Tests:")

# Sentiment Test
sentiment_result = nlp_suite.analyze_sentiment("I love this modern NLP approach!")
print(f"Sentiment: {sentiment_result}")

# QA Test
context = "Transformers are a type of neural network architecture that has revolutionized natural language processing."
question = "What has revolutionized NLP?"
qa_result = nlp_suite.answer_question(context, question)
print(f"QA: {qa_result}")

### 🌐 NLP API Service

In [ ]:
# 🚀 FastAPI NLP Service

from typing import List, Optional

# Pydantic Models für API
class TextGenerationRequest(BaseModel):
    prompt: str
    max_length: Optional[int] = 100
    temperature: Optional[float] = 0.7
    num_sequences: Optional[int] = 1

class SentimentRequest(BaseModel):
    text: str

class QARequest(BaseModel):
    question: str
    context: str

# FastAPI App
app = FastAPI(
    title="Modern NLP API",
    description="Production-ready NLP Services mit Transformers",
    version="2.0.0"
)

# Global Services
global_text_gen = None
global_nlp_suite = None

@app.on_event("startup")
async def startup_event():
    global global_text_gen, global_nlp_suite
    global_text_gen = text_gen
    global_nlp_suite = nlp_suite

@app.get("/")
async def root():
    return {
        "message": "Modern NLP API",
        "version": "2.0.0",
        "services": ["text-generation", "sentiment", "qa"],
        "timestamp": datetime.now().isoformat()
    }

@app.post("/generate")
async def generate_text(request: TextGenerationRequest):
    """Text Generation Endpoint"""
    if not global_text_gen:
        return {"error": "Text Generator nicht verfügbar"}
    
    generated = global_text_gen.generate_text(
        prompt=request.prompt,
        max_length=request.max_length,
        temperature=request.temperature,
        num_return_sequences=request.num_sequences
    )
    
    return {
        "prompt": request.prompt,
        "generated_texts": generated,
        "timestamp": datetime.now().isoformat()
    }

@app.post("/sentiment")
async def analyze_sentiment(request: SentimentRequest):
    """Sentiment Analysis Endpoint"""
    if not global_nlp_suite:
        return {"error": "NLP Suite nicht verfügbar"}
    
    result = global_nlp_suite.analyze_sentiment(request.text)
    
    return {
        "text": request.text,
        "sentiment": result,
        "timestamp": datetime.now().isoformat()
    }

@app.post("/qa")
async def question_answering(request: QARequest):
    """Question Answering Endpoint"""
    if not global_nlp_suite:
        return {"error": "NLP Suite nicht verfügbar"}
    
    result = global_nlp_suite.answer_question(request.context, request.question)
    
    return {
        "question": request.question,
        "context": request.context[:100] + "...",
        "answer": result,
        "timestamp": datetime.now().isoformat()
    }

print("✅ Modern NLP API erstellt!")
print("📝 Endpoints:")
print("   POST /generate - Text Generation")
print("   POST /sentiment - Sentiment Analysis")
print("   POST /qa - Question Answering")
print("\n🚀 Starten mit: uvicorn main:app --reload")

### 📱 Streamlit NLP Dashboard

In [ ]:
# 📊 Streamlit NLP Dashboard erstellen

streamlit_nlp_app = '''
import streamlit as st
import requests
import json
from datetime import datetime

st.set_page_config(
    page_title="Modern NLP Dashboard",
    page_icon="🤖",
    layout="wide"
)

st.title("🤖 Modern NLP Dashboard")
st.markdown("**Transformer-basierte NLP Services**")

# Sidebar
st.sidebar.header("⚙️ Konfiguration")
api_url = st.sidebar.text_input("NLP API URL", "http://localhost:8000")

# Main Content
tab1, tab2, tab3 = st.tabs(["✍️ Text Generation", "😊 Sentiment", "❓ Q&A"])

with tab1:
    st.header("✍️ Text Generation")
    
    col1, col2 = st.columns([2, 1])
    
    with col1:
        prompt = st.text_area("Prompt eingeben:", "Once upon a time")
        
    with col2:
        max_length = st.slider("Max Length", 50, 200, 100)
        temperature = st.slider("Temperature", 0.1, 1.0, 0.7)
        
    if st.button("🚀 Generate Text", type="primary"):
        with st.spinner("Generiere Text..."):
            try:
                response = requests.post(
                    f"{api_url}/generate",
                    json={
                        "prompt": prompt,
                        "max_length": max_length,
                        "temperature": temperature
                    }
                )
                
                if response.status_code == 200:
                    result = response.json()
                    st.success("Text erfolgreich generiert!")
                    st.write("**Generated Text:**")
                    st.write(result["generated_texts"][0])
                else:
                    st.error(f"API Fehler: {response.status_code}")
            except Exception as e:
                st.error(f"Verbindungsfehler: {str(e)}")

with tab2:
    st.header("😊 Sentiment Analysis")
    
    text_input = st.text_area("Text für Sentiment Analysis:")
    
    if st.button("🔍 Analyze Sentiment", type="primary"):
        if text_input:
            with st.spinner("Analysiere Sentiment..."):
                try:
                    response = requests.post(
                        f"{api_url}/sentiment",
                        json={"text": text_input}
                    )
                    
                    if response.status_code == 200:
                        result = response.json()
                        sentiment = result["sentiment"]
                        
                        col1, col2 = st.columns(2)
                        with col1:
                            st.metric("Sentiment", sentiment["label"])
                        with col2:
                            st.metric("Confidence", f"{sentiment['confidence']:.2%}")
                            
                        # Emoji basierend auf Sentiment
                        emoji = "😊" if sentiment["label"] == "POSITIVE" else "😔"
                        st.write(f"## {emoji} {sentiment['label']}")
                        
                    else:
                        st.error(f"API Fehler: {response.status_code}")
                except Exception as e:
                    st.error(f"Verbindungsfehler: {str(e)}")
        else:
            st.warning("Bitte Text eingeben")

with tab3:
    st.header("❓ Question Answering")
    
    context = st.text_area("Kontext:", height=150)
    question = st.text_input("Frage:")
    
    if st.button("💡 Answer Question", type="primary"):
        if context and question:
            with st.spinner("Beantworte Frage..."):
                try:
                    response = requests.post(
                        f"{api_url}/qa",
                        json={
                            "context": context,
                            "question": question
                        }
                    )
                    
                    if response.status_code == 200:
                        result = response.json()
                        answer = result["answer"]
                        
                        st.success("Antwort gefunden!")
                        st.write(f"**Antwort:** {answer['answer']}")
                        st.write(f"**Confidence:** {answer['confidence']:.2%}")
                        
                    else:
                        st.error(f"API Fehler: {response.status_code}")
                except Exception as e:
                    st.error(f"Verbindungsfehler: {str(e)}")
        else:
            st.warning("Bitte Kontext und Frage eingeben")

# Footer
st.divider()
st.caption(f"Modern NLP Dashboard - {datetime.now().strftime('%H:%M:%S')}")
'''

# Streamlit App speichern
with open('07_02_streamlit_nlp_dashboard.py', 'w') as f:
    f.write(streamlit_nlp_app)

print("📱 Streamlit NLP Dashboard erstellt!")
print("📄 Datei: 07_02_streamlit_nlp_dashboard.py")
print("\n🚀 Starten mit: streamlit run 07_02_streamlit_nlp_dashboard.py")

## 🎯 Portfolio-Zusammenfassung: Modern NLP

### 🏆 Erlernte Fähigkeiten

1. **🤗 Transformer Models** - State-of-the-art NLP mit Hugging Face
2. **🔤 Multi-Task NLP** - Text Generation, Sentiment, Q&A
3. **🌐 NLP APIs** - Production-ready Services
4. **📱 Interactive Dashboards** - User-friendly NLP Interfaces
5. **🚀 Modern Deployment** - Skalierbare NLP-Anwendungen

### 📁 Generierte Artefakte

- **🤖 NLP Service Classes** - Wiederverwendbare Components
- **🌐 FastAPI Application** - REST API für NLP Tasks
- **📱 Streamlit Dashboard** - Interactive NLP Interface

### 🚀 Projektideen

1. **📰 News Summarizer** - Automatische Artikel-Zusammenfassungen
2. **💬 Chatbot Service** - Q&A System für Websites
3. **📊 Social Media Monitor** - Sentiment Tracking
4. **✍️ Creative Writing Assistant** - Text Generation Tool

---

> 💡 **Portfolio-Tipp**: Modern NLP zeigt Ihre Kenntnisse aktueller AI-Technologien!